In [1]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display

In [2]:
# Import model modules
from duopoly.duopoly import *
from duopoly.solvers import *

In [3]:
# Create the toggle button
show_f = widgets.ToggleButton(value=False, description='Density Heatmap')
toggle_button = widgets.ToggleButton(value=False, description='Density Heatmap')

def plot_margins(show_f=show_f, xmin=0, xmax=2.5, 
                 t1=1, t2=1, max_λ=5, k=1, μ_λ=0, V_λ=1, N=100_000, γ1=1.5, γ2=1, 
                 p=10, seed=42, α=1, β=1, beta_ind=0):
    
    # Create duopoly object
    M = Duopoly( t1=t1,    # Transport cost
                t2=t2,
                 p=p,    # Price
                 γ1=γ1,  # Productivity of 1
                 γ2=γ2,    # Productivity of 2
                 μ_λ=μ_λ,   # Mean of the normal distribution of log(λ)
                 V_λ=V_λ,   # Variance of the normal distribution of log(λ)
                 k=k,     # Scale parameter for joint distribution
                 α=α,     # Beta distribution parameter
                 β=β,     # Beta distribution parameter
                 beta_ind=beta_ind, # Set to 1 for independent beta draws with parameters α and β
                 N = N, # Number of Monte Carlo draws
                 seed=seed  
                 )
    # Obtain equilibrium values
    sol = solve_duopoly(M, method='broyden1', x1_0=2, x2_0=1.9)
    x1_eq = sol['x1']
    x2_eq = sol['x2']
    q1_eq = sol['q1']
    q2_eq = sol['q2']
    π1_eq = sol['π1']
    π2_eq = sol['π2']

    # Generate x values
    b = np.linspace(0, 1, 100)
    x1_ls = np.linspace(xmin, xmax, 100)
    x2_ls = np.linspace(xmin, xmax, 100)

        
    # Create arrays for graphs
    λ_S = np.array([M.λ_tilde(x1_eq, x2_eq, i) for i in b])
    π1 = np.array([M.Π(1, x, x2_eq) for x in x1_ls])
    π2 = np.array([M.Π(2, x1_eq, x) for x in x2_ls])
    foc1 = np.array([M.FOC(1, x, x2_eq) for x in x1_ls])
    foc2 = np.array([M.FOC(2, x1_eq, x) for x in x2_ls])
    
    # Calculate quantities as a function of x1 and x2
    q1 = M.q(1, x1_eq, x2_eq)
    q2 = M.q(2, x1_eq, x2_eq)

    # Create the plot
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(10, 4))
    fig.suptitle('Duopoly with hetergoenous health and γ1 > γ2')

    # Add the heatmap
    if show_f:
        # Create the heatmap
        ax1.hist2d(M.θ[:, 1], 
                  M.θ[:, 0], 
                  bins=(100, 100), 
                  range=[[0, 1], [0, max_λ]], 
                  cmap='hot', alpha=0.5)
    
    # Plot the switching margin
    ax1.plot(b, λ_S, label='Switching margin', color='#39FF14')
    ax1.set_ylim(0, max_λ)
    ax1.set_xlabel('b')
    ax1.set_ylabel('λ')
    ax1.legend(loc='upper right')
    ax1.grid(True)

    # Plot quantitites in bars for each insurer (show number on top of bars)
    ax2.bar(['q1', 'q2'], [q1, q2])
    ax2.text(0, q1+.05, round(q1, 2), ha='center')
    ax2.text(1, q2+.05, round(q2, 2), ha='center')
    ax2.set_ylabel('Quantity')
    ax2.set_xlabel('Insurer')
    ax2.set_ylim(0, 1)
    ax2.grid(False)
    

    # Plot the profit function for 1
    ax3.plot(x1_ls, π1, label='π1')
    ax3.axvline(x1_eq, color='r', linestyle='--')
    ax3.set_xlabel('x1')
    ax3.set_ylabel('π1(x1 | x2_eq)')
    ax3.legend(loc='upper right')
    ax3.grid(True)

    # Plot FOC1
    #ax4.plot(x1_ls, foc1, label='FOC1')
    #ax4.set_xlabel('x1')
    #ax4.set_ylabel('FOC1(x1 | x2)')
    #ax4.legend(loc='upper right')
    #ax4.grid(True)
    
    # Plot the profit function for 2
    ax4.plot(x2_ls, π2, label='π2')
    ax4.axvline(x2_eq, color='r', linestyle='--')
    ax4.set_xlabel('x2')
    ax4.set_ylabel('π2(x2 | x1_eq)')
    ax4.legend(loc='upper right')
    ax4.grid(True)

    # Plot
    plt.show()

In [4]:
interactive_plot = interactive(plot_margins,
                               p=(0, 10, 0.01),
                               V_λ=(0, 10, 0.01), 
                               x1=(0, 2, 0.01), 
                               x2=(0, 2, 0.01), 
                               t1=(0, 10, .01),
                               t2=(0, 10, .01),
                               k=(0, 2, .01), 
                               max_λ=(0, 10, 0.1), 
                               N=(100_000, 1_000_000, 50_000),
                               γ1=(0, 10, 0.01),
                               γ2=(0, 10, 0.01),
                               α=(0, 10, 0.01),
                               β=(0, 10, 0.01), 
                               xmin=(0, 3, 0.01),
                               μ_λ=(0, 3, 0.01),
                               xmax=(0, 3, 0.01)
                            )

interactive_plot

interactive(children=(ToggleButton(value=False, description='Density Heatmap'), FloatSlider(value=0.0, descrip…